# Imports

In [1]:
import os
import math
import json
import collections

import pandas as pd
import numpy as np

from src.pre_processing.macros import column_groups, DATA_PATH, DATA_SPLIT_PATH, ORIGINAL_DATASET_NAME, DATA_PREPROC_PATH, RES_PATH

# Load Original

In [76]:
df = pd.read_csv(os.path.join(DATA_PATH, ORIGINAL_DATASET_NAME), low_memory=False)
df = df.set_index("id_student")
df

,id_student_original,id_year,id_grade,id_class_group,id_school,id_student_16_19,id_school_16_19,student_questionnaire,principals_questionnaire,family_questionnaire,...,p331a,p331b,p331c,p331d,p331e,p331f,p331g,p331j,pfc,rep
id_student,,,,,,,,,,,,,,,,,,,,,
1,33613.0,2016,3,A,2415.0,14374.0,569.0,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19294.0,2016,3,A,1842.0,8001.0,273.0,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19587.0,2016,3,NaN,1432.0,8142.0,82.0,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29985.0,2016,3,A,2280.0,12800.0,505.0,1,1,1,...,2.0,2.0,3.0,2.0,3.0,3.0,NaN,NaN,NaN,NaN
5,6982.0,2016,3,A,2040.0,2606.0,390.0,1,1,1,...,4.0,NaN,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83853,2500.0,2019,6,B,1278.0,NaN,271.0,1,1,0,...,4.0,4.0,4.0,4.0,4.0,NaN,5.0,3.0,10.0,2.0
83854,5784.0,2019,6,A,1079.0,NaN,78.0,1,1,1,...,3.0,4.0,4.0,4.0,4.0,NaN,5.0,4.0,3.0,2.0
83855,7708.0,2019,6,X,1486.0,3036.0,459.0,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
# Import ranges of each feature
with open(os.path.join(DATA_PATH, "meta_data_original.json")) as file:
    meta_data = json.load(file)

In [116]:
# Check each features is in range
for column_group in meta_data["groups"]:
    for col in column_group["columns"]:
        value = column_group["values"][0]
        if value != ["Unknown"]:
            if type(value) == dict:
                col_min = df[col].min()
                col_max = df[col].max()
                if col_min < int(value["min"]) or col_max > int(value["max"]):
                    print(f""""NO IN LIMIT\n\tcol:{col}, col_min:{col_min}, col_max:{col_max}, min_value:{value["min"]}, max_value:{value["max"]} """)
            elif type(column_group["values"][0]) == list:
                distinct_in_col = [elem for elem in df[col].unique() if type(elem) == str or not math.isnan(elem)]
                value = value if not value[0].isdigit() else [int(elem) for elem in value]
                # if distinct_in_col not in value:
                if not set(distinct_in_col).issubset(set(value)):
                    print(f""""NO IN LIST\n\tcol:{col}, distinct_in_col:{distinct_in_col}, value:{value}""")


"NO IN LIMIT
	col:ESCS, col_min:-2.706281, col_max:3.293104, min_value:1, max_value:25 
"NO IN LIST
	col:p331g, distinct_in_col:[1.0, 3.0, 4.0, 5.0, 2.0], value:[1, 2, 3, 4]
"NO IN LIST
	col:p331j, distinct_in_col:[4.0, 3.0, 5.0, 2.0, 1.0], value:[1, 2, 3, 4]


In [55]:
import 
collections.Counter(df["a1"].unique().tolist()[:2]) == collections.Counter(["2", "1"])

False

[2.0, 1.0]

# Split and Save

In [2]:
# identifiers pre-processing, i.e., change float columns to int
int_identifiers = [col for col in column_groups["identifiers"] if col not in ["id_class_group"]]
df[int_identifiers] = df[int_identifiers].astype('Int64')
df

NameError: name 'df' is not defined

In [5]:
for group, columns in column_groups.items():
    print(group, len(columns))

identifiers 7
info 6
student_scores 6
student_questionnaire 154
school_principals_questionnaire 138
family_questionnaire 91
teacher_questionnaire 158


In [6]:
for group, columns in column_groups.items():
    df[columns].to_csv(os.path.join(DATA_SPLIT_PATH, f"{group}.csv"))

# Load Pre-processed

In [14]:
df = pd.read_csv(os.path.join(DATA_PREPROC_PATH, "normalized.csv"), low_memory=False)
df = df.set_index("id_student")
print(df.shape)
df

(80174, 147)


,s_gender,s_birth_year,s_is_living_with_mother,s_is_living_with_father,s_is_living_with_siblings,s_is_living_with_other_relatives,s_has_repeated,s_frequency_of_skips,s_homeworks_day_per_week,s_extent_of_teachers_affinity,...,t_extent_of_student_involvement_during_class,t_extent_of_teaching_methods_variety,t_extent_of_opinion_on_school,t_extent_of_class_behaviour,t_extent_of_resource_variety,t_extent_of_good_work_by_non_teachers,t_number_of_individual_training_topics,t_number_of_subjects_taught,t_extent_of_satisfaction_job_and_school,t_extent_of_results_satisfaction
id_student,,,,,,,,,,,,,,,,,,,,,
1,FEMALE,2007.0,True,True,NaN,True,False,1.0,0.0,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MALE,2007.0,True,True,True,NaN,True,1.0,3.0,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FEMALE,2007.0,True,True,True,NaN,False,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MALE,2007.0,True,True,True,NaN,False,1.0,3.0,1.000000,...,0.555556,0.666667,1.000000,0.500000,0.458333,0.555556,NaN,NaN,NaN,0.666667
5,FEMALE,2007.0,True,True,True,NaN,False,1.0,1.0,1.000000,...,0.666667,0.733333,1.000000,0.833333,0.625000,0.888889,1.0,NaN,NaN,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83853,FEMALE,NaN,NaN,NaN,NaN,NaN,True,1.0,2.0,1.000000,...,0.555556,0.800000,1.000000,0.500000,0.416667,1.000000,1.0,NaN,1.000000,0.166667
83854,FEMALE,NaN,NaN,NaN,NaN,NaN,False,1.0,4.0,0.333333,...,0.666667,0.733333,0.666667,0.666667,0.500000,1.000000,1.0,NaN,0.722222,0.500000
83855,MALE,NaN,NaN,NaN,NaN,NaN,False,1.0,4.0,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
ids = pd.read_csv(
    os.path.join(DATA_SPLIT_PATH, "identifiers.csv"), low_memory=False
)
ids = ids.set_index("id_student")
int_identifiers = [col for col in ids.columns if col not in ["id_class_group"]]
ids[int_identifiers] = ids[int_identifiers].astype("Int64")
ids

,id_student_original,id_year,id_grade,id_class_group,id_school,id_student_16_19,id_school_16_19
id_student,,,,,,,
1,33613,2016,3,A,2415,14374,569
2,19294,2016,3,A,1842,8001,273
3,19587,2016,3,NaN,1432,8142,82
4,29985,2016,3,A,2280,12800,505
5,6982,2016,3,A,2040,2606,390
...,...,...,...,...,...,...,...
83853,2500,2019,6,B,1278,<NA>,271
83854,5784,2019,6,A,1079,<NA>,78
83855,7708,2019,6,X,1486,3036,459


In [25]:
# Load identifiers and change float columns to int
scores = pd.read_csv(
    os.path.join(DATA_SPLIT_PATH, "student_scores.csv"), low_memory=False
)
scores = scores.set_index("id_student")
# int_identifiers = [col for col in ids.columns if col not in ["id_class_group"]]
# ids[int_identifiers] = ids[int_identifiers].astype('Int64')
scores

,score_MAT,level_MAT,score_LEN,level_LEN,score_ING,level_ING
id_student,,,,,,
1,564.8700,3.0,535.1500,3.0,NaN,NaN
2,388.3400,1.0,293.7000,1.0,NaN,NaN
3,386.5900,1.0,514.8100,3.0,NaN,NaN
4,487.7600,2.0,449.2500,2.0,NaN,NaN
5,709.7900,4.0,598.7200,3.0,NaN,NaN
...,...,...,...,...,...,...
83853,400.8625,2.0,446.6522,2.0,294.7474,1.0
83854,597.0243,3.0,632.6043,4.0,633.2966,4.0
83855,707.9254,4.0,400.2761,2.0,477.5056,2.0


In [26]:
ids = pd.merge(ids, scores, left_index=True, right_index=True)
df = pd.merge(ids, df, left_index=True, right_index=True)
df

,id_student_original,id_year,id_grade,id_class_group,id_school,id_student_16_19,id_school_16_19,score_MAT,level_MAT,score_LEN,...,t_extent_of_student_involvement_during_class,t_extent_of_teaching_methods_variety,t_extent_of_opinion_on_school,t_extent_of_class_behaviour,t_extent_of_resource_variety,t_extent_of_good_work_by_non_teachers,t_number_of_individual_training_topics,t_number_of_subjects_taught,t_extent_of_satisfaction_job_and_school,t_extent_of_results_satisfaction
id_student,,,,,,,,,,,,,,,,,,,,,
1,33613,2016,3,A,2415,14374,569,564.8700,3.0,535.1500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19294,2016,3,A,1842,8001,273,388.3400,1.0,293.7000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19587,2016,3,NaN,1432,8142,82,386.5900,1.0,514.8100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29985,2016,3,A,2280,12800,505,487.7600,2.0,449.2500,...,0.555556,0.666667,1.000000,0.500000,0.458333,0.555556,NaN,NaN,NaN,0.666667
5,6982,2016,3,A,2040,2606,390,709.7900,4.0,598.7200,...,0.666667,0.733333,1.000000,0.833333,0.625000,0.888889,1.0,NaN,NaN,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83853,2500,2019,6,B,1278,<NA>,271,400.8625,2.0,446.6522,...,0.555556,0.800000,1.000000,0.500000,0.416667,1.000000,1.0,NaN,1.000000,0.166667
83854,5784,2019,6,A,1079,<NA>,78,597.0243,3.0,632.6043,...,0.666667,0.733333,0.666667,0.666667,0.500000,1.000000,1.0,NaN,0.722222,0.500000
83855,7708,2019,6,X,1486,3036,459,707.9254,4.0,400.2761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
final = pd.read_csv(
    os.path.join(DATA_PREPROC_PATH, "final.csv"), low_memory=False
)
final

,id_questionnaire,id_student_original,id_year,id_grade,id_class_group,id_school,id_student_16_19,id_school_16_19,score_MAT,level_MAT,...,t_extent_of_student_involvement_during_class,t_extent_of_teaching_methods_variety,t_extent_of_opinion_on_school,t_extent_of_class_behaviour,t_extent_of_resource_variety,t_extent_of_good_work_by_non_teachers,t_number_of_individual_training_topics,t_number_of_subjects_taught,t_extent_of_satisfaction_job_and_school,t_extent_of_results_satisfaction
0,1,33613.0,2016,3,A,2415.0,14374.0,569.0,564.8700,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,19294.0,2016,3,A,1842.0,8001.0,273.0,388.3400,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,19587.0,2016,3,NaN,1432.0,8142.0,82.0,386.5900,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,29985.0,2016,3,A,2280.0,12800.0,505.0,487.7600,2.0,...,0.555556,0.666667,1.000000,0.500000,0.458333,0.555556,NaN,NaN,NaN,0.666667
4,5,6982.0,2016,3,A,2040.0,2606.0,390.0,709.7900,4.0,...,0.666667,0.733333,1.000000,0.833333,0.625000,0.888889,1.0,NaN,NaN,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80169,83853,2500.0,2019,6,B,1278.0,NaN,271.0,400.8625,2.0,...,0.555556,0.800000,1.000000,0.500000,0.416667,1.000000,1.0,NaN,1.000000,0.166667
80170,83854,5784.0,2019,6,A,1079.0,NaN,78.0,597.0243,3.0,...,0.666667,0.733333,0.666667,0.666667,0.500000,1.000000,1.0,NaN,0.722222,0.500000
80171,83855,7708.0,2019,6,X,1486.0,3036.0,459.0,707.9254,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80172,83856,18965.0,2019,6,B,1036.0,9913.0,36.0,522.8511,3.0,...,0.666667,0.666667,0.666667,0.666667,0.458333,0.625000,3.0,NaN,0.611111,0.500000


In [30]:
list(final.columns)

['id_questionnaire',
 'id_student_original',
 'id_year',
 'id_grade',
 'id_class_group',
 'id_school',
 'id_student_16_19',
 'id_school_16_19',
 'score_MAT',
 'level_MAT',
 'score_LEN',
 'level_LEN',
 'score_ING',
 'level_ING',
 's_gender',
 's_birth_year',
 's_is_living_with_mother',
 's_is_living_with_father',
 's_is_living_with_siblings',
 's_is_living_with_other_relatives',
 's_has_repeated',
 's_frequency_of_skips',
 's_homeworks_day_per_week',
 's_extent_of_teachers_affinity',
 's_birth_country',
 's_nazionality_country',
 's_weight',
 's_frequency_of_computer_usage',
 's_frequency_of_internet_usage',
 's_frequency_of_work_with_teachers',
 's_frequency_of_materials_in_class',
 's_frequency_of_evaluations',
 's_extent_of_teacher_performance',
 's_extent_of_class_env',
 's_extent_of_classmates_affinity',
 's_extent_of_school_satisfaction',
 's_extent_of_math_affinity',
 's_extent_of_reading_affinity',
 'p_gender',
 'p_number_of_abroad_students_country_no_spanish',
 'p_number_of_abr

In [39]:
# Import meta data of merged
with open(os.path.join(RES_PATH, "meta_data_final.json")) as file:
    meta_data = json.load(file)

In [31]:
meta_list = list(meta_data.keys())
df_list = list(final.columns)
for elem in df_list:
    if elem not in meta_list:
        print(elem)

id_questionnaire
id_student_original
id_year
id_grade
id_class_group
id_school
id_student_16_19
id_school_16_19
score_MAT
level_MAT
score_LEN
level_LEN
score_ING
level_ING


In [35]:
# Write
# with open(os.path.join(DATA_PATH, "meta_data_new_norm_py.json"), 'w') as f:
#     f.write(json.dumps(meta_data))

In [37]:
df = final

In [41]:
# Check each features is in range
for col, details in meta_data.items():
    # print(col)
    value = details["values"][0][0]
    # if value == ["1", "2", "3", "4"]:
    #     value = {"min": 1, "max": 4}
    if value not in [["Unknown"], ["Integer"], ["Float"]]:
        if type(value) == dict:
            try:
                col_min = df[col].min()
                col_max = df[col].max()
                if col_min < int(value["min"]) or col_max > int(value["max"]):
                    print(f""""NO IN LIMIT\n\tcol:{col}, col_min:{col_min}, col_max:{col_max}, min_value:{value["min"]}, max_value:{value["max"]} """)
            except:
                print(col)
                print([elem for elem in df[col].unique() if type(elem) == str or not math.isnan(elem)])
        elif type(value) == list:
            distinct_in_col = [elem for elem in df[col].unique() if type(elem) == str or not math.isnan(elem)]
            value = value if not value[0].isdigit() else [int(elem) for elem in value]
            if "True" not in value:
                # if distinct_in_col not in value:
                if not set(distinct_in_col).issubset(set(value)):
                    print(f""""NO IN LIST\n\tcol:{col}, distinct_in_col:{distinct_in_col}, value:{value}""")
